In [ ]:
import random
import re


def assemble_snippet(essays, anchor_phrase, relative_anchor_pos, target_length):
    random.shuffle(essays)
    sentences = re.split(r'(?<=[.!?])\s+', " ".join(essays))
    if not sentences:
        return ""
    
    l = r = random.randint(0, len(sentences) - 1)
    total_length = lambda l, r: sum(len(s) for s in sentences[l:r+1]) + (r - l)
    
    while total_length(l, r) < target_length:
        if l > 0:
            l -= 1
        if total_length(l, r) >= target_length:
            break
        if r < len(sentences) - 1:
            r += 1
        if l == 0 and r == len(sentences) - 1:
            break

    window = sentences[l:r+1]
    insert_index = int(relative_anchor_pos * len(window))
    window.insert(insert_index, anchor_phrase)
    return " ".join(window).strip()


In [ ]:

from datasets import load_dataset
essays = [essay["text"] for essay in load_dataset('sgoel9/paul_graham_essays', split='train')]

In [ ]:

from openai import OpenAI

anchor = "The RAG Tutorial at the Weizenbaum Institute takes place on the 7th of March."
question = "What is the date of the RAG Tutorial at the Weizenbaum Institute?"

target_prompt_length = 100000

result = []
num_tokens = []

client = OpenAI()
for pos in [0.1 * i for i in range(11)]:
    context = ""
    context = assemble_snippet(essays, anchor, pos, target_prompt_length * 4) # factor 4 because the assembly functions counts chars, not tokens
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": f"{question}\n\n<context>{context}</context>"}],
        max_tokens=100
    )
    answer = response.choices[0].message.content
    num_tokens.append(response.usage.prompt_tokens)
    correct = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": f"Only answer with True or False"},
            {"role": "user", "content": f"Does the answer contain the expected answer for the questions '{question}'\n<provided-answer>{answer}</provided-answer>\n<correct-answer>{anchor}</correct-answer>"}
        ],
        max_tokens=100
    )
    result.append(bool(correct.choices[0].message.content))

print(sum(result) / len(result))

In [ ]:
num_tokens